In [1]:
import pandas as pd

In [2]:
file = pd.read_csv('2020-03-03_all_atm_temp.csv')

In [3]:
#Cleans up the plot list that was in a list so it can be manipulated
def clean_alt_list(list_):
    list_ = list_.replace('[', '')
    list_ = list_.replace(']', '')
    return list_

In [4]:
file['plot'] = (file['plot'].apply(clean_alt_list)).apply(eval)

In [8]:
#Takes each image (with multiple plots per row) and breaks it up so you have all of the plots listed out with associated data
def expand_plots(clean_file):
    plot_expand = file['plot'].apply(pd.Series)
    plot_expand['time'] = file['Date and Time']
    plot_expand['Image Name'] = file['Image Name']
    plot_expand['env_temp'] = file['env_temp']
    plot_expand['env_wind'] = file['env_wind']
    plot_expand['azmet_atm_temp'] = file['azmet_atm_temp']
    plot_expand['azmet_wind_velocity'] = file['azmet_wind_velocity']
    plot_expand['azmet_VPD'] = file['azmet_VPD']
    plot_expand['azmet_solar_radiation'] = file['azmet_solar_radiation']
    #plot_expand['Env Temp avg'] = file['Avg EL']
    #plot_expand['Azmet Temp avg'] = file['Avg AZmet']
    plot_expand['relative_humidity'] = file['relative_humidity']
    stacked = plot_expand.set_index(['time', 'Image Name', 'env_temp', 'env_wind', 'azmet_atm_temp', 'azmet_wind_velocity',
                                     'azmet_VPD', 'azmet_solar_radiation', 'relative_humidity']).stack()
    stack_df = pd.DataFrame(stacked).reset_index()
    del stack_df['level_9']
    final_df = stack_df.rename(columns = {0:'Plot'})
    return final_df
#Takes a plot as paramter and finds how long it took to image that plot
def plot_scan_time(interest_plot):
    final_df = expand_plots(file)
    plot_duration = final_df.loc[final_df['Plot'] == interest_plot]
    start = pd.to_datetime(plot_duration['time'].iloc[(0)])
    end = pd.to_datetime(plot_duration['time'].iloc[(-1)])
    return end - start
#Takes a plot as parameter and finds how much the temperature changed during the time it took to image the plot
def plot_temp_change(interest_plot):
    final_df = expand_plots(file)
    plot_duration = final_df.loc[final_df['Plot'] == interest_plot]
    start = plot_duration['Env Temp'].iloc[(0)]
    end = plot_duration['Env Temp'].iloc[(-1)]
    return end - start
def images_making_plot(interest_plot):
    final_df = expand_plots(file)
    plot_list = final_df.loc[final_df['Plot'] == interest_plot]
    return plot_list

In [67]:
#plot_scan_time('MAC Field Scanner Season 10 Range 25 Column 10')

In [68]:
#plot_temp_change('MAC Field Scanner Season 10 Range 25 Column 10')

In [69]:
#images_making_plot('MAC Field Scanner Season 10 Range 25 Column 10')

In [9]:
finale_df = expand_plots(file)

In [10]:
finale_df.to_csv('2020-03-03_env.csv')

In [11]:
finale_df_mod = finale_df[['Image Name', 'Plot']]

In [12]:
finale_df_mod.to_csv('finale_df_mod.csv')